**STEPS TO FOLLOW**

- Pick the Active Eligible table from config Table where active_flag=1, load_flag=1 and bq_to_gcs_status is 'COMPLETED'
- Reads the parqut file from GCS (GCS path is like gcs_path/dt=/*.parquet) means take all the files present inside gcs path for any particular timestamp 
- Uses the latest dt(datetime file) folder
- Writes the delta bronze path
- Create the Table from bronze path to bronze dataset 
- Update the config table

In [0]:
dbutils.widgets.text("mysql_host","34.56.24.205")
mysql_host = dbutils.widgets.get("mysql_host")

dbutils.widgets.text("mysql_port","3306")
mysql_port = dbutils.widgets.get("mysql_port")

dbutils.widgets.text("mysql_root","root")
mysql_user = dbutils.widgets.get("mysql_root")

dbutils.widgets.text("mysql_db","GCPMigrationMeta")
mysql_db = dbutils.widgets.get("mysql_db")

dbutils.widgets.text("mysql_password","Anupamnaina#18")
mysql_password = dbutils.widgets.get("mysql_password")

dbutils.widgets.text("bq_sa_key",'/Volumes/workspace/default/csv/datamigrationproject-483310-739969975183.json')
GOOGLE_APPLICATION_CREDENTIALS = dbutils.widgets.get("bq_sa_key")

dbutils.widgets.text("hive_db","bronze")
hive_db = dbutils.widgets.get("hive_db")

In [0]:
%pip install mysql-connector-python google-cloud-bigquery google-cloud-storage

In [0]:
import json, os, datetime as dt
import mysql.connector as mc
from google.cloud import bigquery
from contextlib import contextmanager 
# contextlib python library help us to setup and clean up the resources automatically

MYSQL = {
    'host':mysql_host.strip(),
    'port':int(mysql_port),
    'user':mysql_user.strip(),
    'pwd':mysql_password,
    'db' :mysql_db
}
BQ_KEY=GOOGLE_APPLICATION_CREDENTIALS.replace('dbfs:/','/dbfs/') 